In [47]:
import pandas as pd
import numpy as np
import pandas_profiling as pp

In [48]:
import os

path = '/home/srgrace/genericContest_data/airplane-accident-severity'

train = pd.read_csv(os.path.join(path, 'train.csv'))
test = pd.read_csv(os.path.join(path, 'test.csv'))

In [49]:
train.sample(5)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
3421,Highly_Fatal_And_Damaging,13.744292,17,3,71.604376,0.290149,75.55,4,27801.684068,2,0.050881,8140
3031,Significant_Damage_And_Serious_Injuries,43.652968,12,10,62.625342,0.419251,78.85,4,39649.807680,3,0.073173,3740
231,Significant_Damage_And_Fatalities,28.401826,13,29,52.552416,0.467574,78.49,4,28416.808851,2,0.051566,10364
3885,Significant_Damage_And_Serious_Injuries,52.922374,9,0,82.087511,0.271757,77.80,3,37541.676373,4,0.187060,12180
2744,Significant_Damage_And_Serious_Injuries,53.835616,5,4,60.027347,0.407711,77.55,6,32266.136231,2,0.008009,544


In [50]:
test.sample(5)

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
1628,68.630137,11,2,71.011851,0.467574,79.76,5,36617.724439,1,0.024697,8052
325,50.776256,10,5,68.778487,0.282576,81.10,6,30479.980245,1,0.007058,1608
1693,38.127854,10,0,63.992707,0.245432,78.65,3,22999.948022,3,0.114141,8345
2187,33.926941,11,4,86.508660,0.351093,80.95,6,25794.580872,1,0.006353,10927
1497,68.721461,11,15,69.006381,0.380304,88.48,2,27975.799439,3,0.374891,7338


In [6]:
pp.ProfileReport(train)

In [51]:
train.Severity.value_counts()

Highly_Fatal_And_Damaging                  3049
Significant_Damage_And_Serious_Injuries    2729
Minor_Damage_And_Injuries                  2527
Significant_Damage_And_Fatalities          1695
Name: Severity, dtype: int64

In [52]:
from sklearn.preprocessing import LabelEncoder

lbl_enc = LabelEncoder()
train['Severity'] = lbl_enc.fit_transform(train['Severity'].values)

In [53]:
train.sample(5)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
1716,1,50.730594,13,0,68.459435,0.399777,79.61,1,33501.232804,2,1.232455,3654
8769,0,44.337900,8,14,61.485871,0.267790,78.02,4,27986.854602,3,0.051656,11394
6962,3,76.392694,10,1,55.925251,0.456395,85.02,7,50713.170165,2,0.004493,6010
5917,0,49.589041,7,7,61.303555,0.476229,79.49,4,40130.086673,2,0.074041,9734
247,0,23.789954,14,7,79.216044,0.324768,78.78,1,13658.816500,2,0.518572,10305


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import pickle
import lightgbm as lgb
from sklearn.decomposition import PCA

x = train.loc[:, ~train.columns.isin(['Accident_ID', 'Severity'])].values

y = train.Severity.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=37)
 

clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=25,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=200, n_jobs=-1, num_leaves=300, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)  # XGBClassifier()

clf.fit(x_train, y_train)
# pickle.dump(clf, open('./data/lgb.pickle', 'wb'))

# clf1 = XGBClassifier()
# clf1.fit(x_train, y_train)

pred = clf.predict(x_test)
# pred1 = clf1.predict(x_test)

print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred, target_names=['0', '1', '2', '3']))

# print(confusion_matrix(y_test, list(map(int, (pred+pred1)/2))))
# print(classification_report(y_test, list(map(int, (pred+pred1)/2)), target_names=['0', '1', '2', '3']))

[[569   7   7   7]
 [  4 490   0   6]
 [  9   3 329   4]
 [  9   6   2 548]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       590
           1       0.97      0.98      0.97       500
           2       0.97      0.95      0.96       345
           3       0.97      0.97      0.97       565

   micro avg       0.97      0.97      0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000



In [73]:
lgb_params = {
    'learning_rate': 0.1,
    'max_depth': 25,
    'num_leaves': 300, 
    'objective': 'multiclass',
    'num_class':4,
    'tree_learner':'voting',
    'metric':'multi_logloss',
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'max_bin': 100}

dtrain_lgb = lgb.Dataset(x_train, label=y_train)

# LightGBM, cross-validation
cv_result_lgb = lgb.cv(lgb_params, 
                       dtrain_lgb, 
                       num_boost_round=1000, 
                       nfold=5, 
                       stratified=True, 
                       early_stopping_rounds=50, 
                       verbose_eval=100, 
                       show_stdv=True)

num_boost_rounds_lgb = len(cv_result_lgb['multi_logloss-mean'])
print('num_boost_rounds_lgb=' + str(num_boost_rounds_lgb))

[100]	cv_agg's multi_logloss: 0.140447 + 0.0477001
num_boost_rounds_lgb=113


In [74]:
model_lgb = lgb.train(lgb_params, dtrain_lgb, num_boost_round=num_boost_rounds_lgb)

pred = model_lgb.predict(x_test)
pred1 = []
for k in pred:
    pred1.append(np.argmax(k))

# print(pred1)

print(confusion_matrix(y_test, pred1))
print(classification_report(y_test, pred1, target_names=['0', '1', '2', '3']))

[[570  10   6   4]
 [  6 488   0   6]
 [  7   1 334   3]
 [ 11   8   6 540]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       590
           1       0.96      0.98      0.97       500
           2       0.97      0.97      0.97       345
           3       0.98      0.96      0.97       565

   micro avg       0.97      0.97      0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000



In [75]:
pred = model_lgb.predict(test.loc[:, ~test.columns.isin(['Accident_ID'])].values)
# pred1 = clf1.predict(test.loc[:, ~test.columns.isin(['Accident_ID'])].values)
# pred2 = list(map(int, (pred+pred1)/2))
pred1 = []
for k in pred:
    pred1.append(np.argmax(k))
    
predict = lbl_enc.inverse_transform(pred1)

submission = pd.DataFrame(
    {'Accident_ID': test['Accident_ID'],
     'Severity': predict,
     })
submission.to_csv('./data/cv_lgb_submission.csv', index=False)

In [69]:
pred = clf.predict(test.loc[:, ~test.columns.isin(['Accident_ID'])].values)
# pred1 = clf1.predict(test.loc[:, ~test.columns.isin(['Accident_ID'])].values)
# pred2 = list(map(int, (pred+pred1)/2))
predict = lbl_enc.inverse_transform(pred)

submission = pd.DataFrame(
    {'Accident_ID': test['Accident_ID'],
     'Severity': predict,
     })
submission.to_csv('./data/lgb_submission.csv', index=False)